Dans ce notebook, on indique la démarche suivie pour créer une base de donnée prête à être analysée à partir des indicateurs issus de la banque mondiale (https://data.worldbank.org/indicator/). On peut reprendre cette démarche pour rajouter ou enlever d'autres indicateurs, et on peut à la fin choisir de ne conserver que certains pays en en mettant la liste. 

La base créée dans ce code comporte ces indicateurs :
    
    -Le pib en parité de pouvoir d'achat en dollars de 2017
    -Le pib par habitant en parité de pouvoir d'achat en dollars de 2017
    -les exportations en parité de pouvoir d'achat en dollars de 2017
    -les exportations en parité de pouvoir d'achat en dollars de 2017 par habitant
    -les exportations en parité de pouvoir d'achat en dollars de 2017 par unité de travail
    -le taux d'exportation en pct du pib 
    -le taux de chomage en pct de la force de travail
    -la population active
    -la population
    -la FBCF en pct du PIB
    -la R&D en pct du pib
    -les taxes payées par les entreprises en pct du profit commercial (détaillé en les trois variables ci-dessous) 
    -le taux d'imposition sur le profit en pct du profit commercial
    -le taux de taxe sur le travail et cotisations sociales en pct du profit commercial
    -les autres taxes payées par les entreprises en pct du profit commercial
    -les IDE entrants en pct du pib

# Code

On commence par importer les modules nécessaires, et on crée quelques fonctions qui vont nous aider à manipuler les dataframe en jonglant entre les codes et les noms des pays

In [1]:
import pandas as pd
import re
import numpy as np

def refresh_dico(Name,Code,df):
    dico = {}
    for i in df[[Name,Code]].dropna().drop_duplicates().iterrows() :
        dico[i[1][Name]] = i[1][Code]
    return (dico)

def rech_ligne_pays (y,df) :
    expression = re.compile(dic_pays[y]+".*?;")
    return(list(i[:-1] for i in (expression.findall(';'.join(df.index)+';'))))

def rech_ligne_indic (y,df):
    expression = re.compile(".{,4}"+y+";")
    return(list(i[:-1] for i in (expression.findall(';'.join(df.index)+';'))))

On importe ensuite les bases au format csv, que l'on est allé chercher sur le site de la banque mondiale qui est en open data (https://data.worldbank.org/indicator/).

Attention, pour ouvrir proprement ces données, il faut d'abord ouvrir les csv dans le bloc notes (ou n'importe quel autre logiciel capable de les ouvrir), et supprimer les deux premières lignes du document qui correspondent plus ou moins au titre du document, mais qui font bugger la fonction pd.read_csv.

In [79]:
#Importation des bases csv, et crétaion de la base aggrégée
gdp = pd.read_csv('pib_cst2017_ppp.csv')
gdp['Simple Indicator'] = 'gdp'

gdp_hab = pd.read_csv('pib_hab_cst2017_ppp.csv')
gdp_hab['Simple Indicator'] = 'gdb_hab'

fbcf = pd.read_csv('FBCF.csv')
fbcf['Simple Indicator'] = 'FBCF'

exports_rate = pd.read_csv('exports_rate.csv')
exports_rate['Simple Indicator'] = 'exports_rate'

unemployment_rate = pd.read_csv('unemployment_rate.csv')
unemployment_rate['Simple Indicator'] = 'unemployment rate'

pop = pd.read_csv('pop.csv')
pop['Simple Indicator'] = 'pop'

labor_force = pd.read_csv('labor force.csv')
labor_force ['Simple Indicator'] = 'labor_force'

ide = pd.read_csv('IDE.csv')
ide ['Simple Indicator'] = 'IDE'

labor_tax_contrib = pd.read_csv('labor_tax_contrib.csv')
labor_tax_contrib ['Simple Indicator'] = 'labor_tax_contrib'

other_taxes_entreprise = pd.read_csv('other_taxes.csv')
other_taxes_entreprise ['Simple Indicator'] = 'other_taxes_entreprise'

profit_tax = pd.read_csv('profit_tax.csv')
profit_tax ['Simple Indicator'] = 'profit_tax'

total_tax_contrib = pd.read_csv('total_tax_contrib.csv')
total_tax_contrib ['Simple Indicator'] = 'total_tax_contrib'

RetD = pd.read_csv('RetD.csv')
RetD ['Simple Indicator'] = 'RetD'



In [80]:
#création de la base aggrégée
df_list = [gdp, gdp_hab, fbcf, unemployment_rate, exports_rate, pop, labor_force, ide, 
           labor_tax_contrib, other_taxes_entreprise, profit_tax, total_tax_contrib, RetD]

for df in df_list :
    df['Pays_indic'] = df['Country Code'].str[:3] + '_' + df['Indicator Name'].str[:99]
df1 = pd.concat(df_list)
df1 = df1.sort_values('Pays_indic').set_index('Pays_indic')
df1.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65,Simple Indicator
Pays_indic,,,,,,,,,,,,,,,,,,,,,
ABW_Exports of goods and services (% of GDP),Aruba,ABW,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,76.509512,77.555556,73.517030,71.294029,73.332115,NaN,NaN,NaN,NaN,exports_rate
"ABW_Foreign direct investment, net inflows (% of GDP)",Aruba,ABW,"Foreign direct investment, net inflows (% of GDP)",BX.KLT.DINV.WD.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,8.378924,9.062755,-0.985625,0.928928,5.315477,NaN,NaN,NaN,NaN,IDE


On supprime d'abord les colonnes qui ne nous servent à rien : le nom de code de l'indicateur, les années que l'on ne conserve pas pour l'étude, et la dernière colonne qui est un bug lors de l'ouverture de ces données avec la fonction pd.read_csv

On fait ensuite de même avec les lignes, en supprimant tous les aggrégats de pays qui ne sont pas proprement des pays

In [81]:
#SUPPRESSION DES LIGNES ET COLONNES INUTILES
dic_pays = refresh_dico('Country Name','Country Code',df1)


useless_data_list = ['Unnamed: 65', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', 
                   '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', 
                   '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', 
                     '2019', '2020']
df1 = df1.drop(useless_data_list, axis = 1)


suppr_liste = ['Arab World', 'Central Europe and the Baltics',
 'Caribbean small states', 'East Asia & Pacific (excluding high income)', 
  'Early-demographic dividend', 
 'East Asia & Pacific',
 'Europe & Central Asia (excluding high income)', 
 'Europe & Central Asia',  
 'European Union','Fragile and conflict affected situations',
 'High income',
 'Heavily indebted poor countries (HIPC)','IBRD only',
 'IDA & IBRD total',
 'IDA total',
 'IDA blend',
 'IDA only',
 'Latin America & Caribbean (excluding high income)',
 'Latin America & Caribbean',
 'Least developed countries: UN classification',
 'Low income',
 'Lower middle income',
 'Low & middle income',
 'Late-demographic dividend',
 'Middle East & North Africa',
 'Middle income',
 'Middle East & North Africa (excluding high income)',
 'North America',
 'OECD members',
 'Other small states',
 'Pre-demographic dividend',
 'Pacific island small states',
 'Post-demographic dividend',
 'South Asia',
 'Sub-Saharan Africa (excluding high income)',
 'Sub-Saharan Africa',
 'Small states',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'South Asia (IDA & IBRD)',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Upper middle income',
 'World',
 'Central African Republic',
 'West Bank and Gaza',
 'Hong Kong SAR, China',
 'Macao SAR, China']
for i in suppr_liste :
    df1 = df1.drop(labels = rech_ligne_pays(i,df1)) 

On supprime maintenant tous les pays pour lesquels il manque au moins un indicateur. On le fait en deux étapes :

(1) On supprime toutes les lignes où il manque au moins une donnée à un moment donné.
(2) On supprime toutes les lignes de tous les pays pour lesquels on a supprimé au moins une ligne

In [82]:
dic_code = refresh_dico('Country Code','Country Name',df1)

for i in rech_ligne_pays(dic_code['USA'],df1):
    print(df1.loc[i])

Country Name                                   United States
Country Code                                             USA
Indicator Name      Exports of goods and services (% of GDP)
2005                                                  10.012
2006                                                 10.6598
2007                                                 11.4923
2008                                                 12.4861
2009                                                 10.9489
2010                                                 12.3151
2011                                                 13.5305
2012                                                 13.5289
2013                                                 13.5445
2014                                                 13.5316
2015                                                 12.4329
2016                                                 11.9005
2017                                                 12.1652
2018                    

In [83]:
#Étape 1
df1 = df1.dropna(axis = 0)
df1.head(7)
print (df1.shape)
#Étape 2 
dic_code = refresh_dico('Country Code','Country Name',df1)

for i in dic_pays.keys() :
    if len(rech_ligne_pays(i,df1)) < len(df_list) :
        df1 = df1.drop(rech_ligne_pays(i,df1))

dic_code = refresh_dico('Country Code','Country Name',df1)

#On affiche le nombre de pays restant dans la base
print(len(dic_code.keys()))

(1507, 18)
50


On recrée maintenant la variable correpondant aux exportations par pays en dollars, à partir du PIB et des exportations en pourcentage du PIB. 

Après l'avoir fait, on peut supprimer l'ancienne variable qui exprime les exportations en pct du PIB en enlevant le # sur la ligne correspondante.

On a utlisé la formule :

    Exportation$ = (Exportations%PIB * PIB)/100

In [56]:
#CREATION DE LA VARIABLE EXPORTATION A PARTIR DE EXPORTATION RATE ET DU PIB

for i in dic_code.keys():
    df1.loc[i+ '_Exports of goods and services, PPP (constant 2017 international $)']= (df1.loc[i+'_Exports of goods and services (% of GDP)'][3:df1.shape[1]-1].astype(np.float64) * df1.loc[i+'_GDP, PPP (constant 2017 international $)'][3:df1.shape[1]-1].astype(np.float64)) / 100
    df1.loc[i+ '_Exports of goods and services, PPP (constant 2017 international $)',['Country Name','Country Code']]=df1.loc[i+'_Exports of goods and services (% of GDP)',['Country Name','Country Code']]
    df1.loc[i+ '_Exports of goods and services, PPP (constant 2017 international $)','Indicator Name']= 'Exports of goods and services, PPP (constant 2017 international $)'
    df1.loc[i+ '_Exports of goods and services, PPP (constant 2017 international $)','Simple Indicator'] = 'exports'
    #df1 = df1.drop(labels = i+'_Exports of goods and services (% of GDP)')
df1 = df1.sort_index()

On va maintenant créer les variables par habitant, et par population active

In [27]:
def gen_per_capita(variable):
    for i in dic_code.keys():
        df1.loc[i+variable+' per capita']= df1.loc[i+variable][3:df1.shape[1]-1].astype(np.float64) / df1.loc[i+'_Population, total'][3:df1.shape[1]-1].astype(np.float64)
        df1.loc[i+variable+' per capita',['Country Name','Country Code']]=df1.loc[i+variable,['Country Name','Country Code']]
        df1.loc[i+variable+' per capita','Indicator Name']= variable + ' per capita'
        df1.loc[i+variable+' per capita','Simple Indicator'] = df1.loc[i+variable, 'Simple Indicator'] + '_hab'
    return
gen_per_capita('_Exports of goods and services, PPP (constant 2017 international $)')

def gen_per_labor_unit(variable) :
    for i in dic_code.keys():
        df1.loc[i+variable+' per labor unit']= df1.loc[i+variable][3:df1.shape[1]-1].astype(np.float64) / df1.loc[i+'_Labor force, total'][3:df1.shape[1]-1].astype(np.float64)
        df1.loc[i+variable+' per labor unit',['Country Name','Country Code']]=df1.loc[i+variable,['Country Name','Country Code']]
        df1.loc[i+variable+' per labor unit','Indicator Name']= variable + ' per labor unit'
        df1.loc[i+variable+' per labor unit','Simple Indicator'] = df1.loc[i+variable, 'Simple Indicator'] + '_labor_unit'
    return
gen_per_labor_unit('_Exports of goods and services, PPP (constant 2017 international $)')
df1 = df1.sort_index()

In [28]:
df1.head(10)

,Country Name,Country Code,Indicator Name,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Simple Indicator
Pays_indic,,,,,,,,,,,,,,,,,,
ARM_Exports of goods and services (% of GDP),Armenia,ARM,Exports of goods and services (% of GDP),2.731320e+01,2.214598e+01,1.818936e+01,1.426573e+01,1.467090e+01,1.974842e+01,2.252319e+01,2.756181e+01,2.835349e+01,2.856038e+01,2.972539e+01,3.374428e+01,3.821605e+01,3.939289e+01,exports_rate
"ARM_Exports of goods and services, PPP (constant 2017 international $)",Armenia,ARM,"Exports of goods and services, PPP (constant 2...",6.041838e+09,5.545464e+09,5.178700e+09,4.341853e+09,3.835580e+09,5.276639e+09,6.300887e+09,8.265600e+09,8.783617e+09,9.166229e+09,9.845414e+09,1.119887e+10,1.363416e+10,1.478483e+10,exports
"ARM_Exports of goods and services, PPP (constant 2017 international $) per capita",Armenia,ARM,"_Exports of goods and services, PPP (constant ...",2.026599e+03,1.874540e+03,1.765897e+03,1.493268e+03,1.328067e+03,1.833874e+03,2.190441e+03,2.865792e+03,3.031359e+03,3.147308e+03,3.365317e+03,3.814140e+03,4.629896e+03,5.008790e+03,exports_hab
"ARM_Exports of goods and services, PPP (constant 2017 international $) per labor unit",Armenia,ARM,"_Exports of goods and services, PPP (constant ...",4.905307e+03,4.492542e+03,4.179897e+03,3.492981e+03,3.061103e+03,4.027366e+03,4.696130e+03,6.118040e+03,6.456880e+03,6.799465e+03,7.376085e+03,8.548359e+03,1.045843e+04,1.134595e+04,exports_labor_unit
"ARM_Foreign direct investment, net inflows (% of GDP)",Armenia,ARM,"Foreign direct investment, net inflows (% of GDP)",5.960106e+00,7.307315e+00,7.252332e+00,8.092349e+00,8.788695e+00,5.716038e+00,6.440668e+00,4.676728e+00,3.111931e+00,3.502114e+00,1.744737e+00,3.164508e+00,2.176847e+00,2.040033e+00,IDE
"ARM_GDP per capita, PPP (constant 2017 international $)",Armenia,ARM,"GDP per capita, PPP (constant 2017 internation...",7.419854e+03,8.464469e+03,9.708405e+03,1.046752e+04,9.052390e+03,9.286181e+03,9.725271e+03,1.039769e+04,1.069131e+04,1.101984e+04,1.132136e+04,1.130307e+04,1.211506e+04,1.271496e+04,gdb_hab
"ARM_GDP, PPP (constant 2017 international $)",Armenia,ARM,"GDP, PPP (constant 2017 international $)",2.212058e+10,2.504050e+10,2.847104e+10,3.043555e+10,2.614413e+10,2.671931e+10,2.797511e+10,2.998932e+10,3.097897e+10,3.209421e+10,3.312123e+10,3.318747e+10,3.567653e+10,3.753171e+10,gdp
ARM_Gross capital formation (% of GDP),Armenia,ARM,Gross capital formation (% of GDP),3.576544e+01,4.222546e+01,4.435162e+01,4.794284e+01,4.114505e+01,3.881475e+01,3.192180e+01,2.468095e+01,2.215249e+01,2.128304e+01,2.073053e+01,1.801551e+01,1.842174e+01,2.239974e+01,FBCF
"ARM_Labor force, total",Armenia,ARM,"Labor force, total",1.231694e+06,1.234371e+06,1.238954e+06,1.243022e+06,1.253006e+06,1.310196e+06,1.341719e+06,1.351021e+06,1.360350e+06,1.348081e+06,1.334775e+06,1.310061e+06,1.303653e+06,1.303093e+06,labor_force


On affiche le nombre de pays, et on vérifie qu'on a bien toujours la France, et on affiche la taille du df.

In [31]:
print('On a conservé ', len(dic_code.keys()), ' pays')
print('La taille du df est de ', df1.shape)
df1.loc[rech_ligne_pays('France', df1)]
dic_code.keys()

On a conservé  41  pays
La taille du df est de  (656, 18)


dict_keys(['ARM', 'AUT', 'AZE', 'BEL', 'BGR', 'BLR', 'CAN', 'COL', 'CZE', 'DEU', 'DNK', 'EGY', 'EMU', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN', 'ISR', 'ITA', 'KAZ', 'KOR', 'KWT', 'LTU', 'LVA', 'MDA', 'MKD', 'MNG', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'SWE', 'UKR', 'UZB'])

In [30]:
df1.to_csv('C:/Users/Titouan/Desktop/WB_extended_data.csv')

###Récupération des bases de codes des noms de pays

In [2]:

import urllib#pour récupérer les données
import bs4#pour rendre lisibles les données

In [3]:
def Scrap (url):
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    page = bs4.BeautifulSoup(html, "lxml")
    return page

In [4]:
def ToDf (table) :
    Listgrossiere=pd.read_html(str(table))#on transforme le tableau en liste de dataframes
    listeDf=Listgrossiere[0]#on récupère le dataframe
    return listeDf

In [10]:
page=Scrap('https://laendercode.net/fr/2-letter-list.html')
table = page.find('table')
liste2=ToDf(table)

In [11]:
page=Scrap('https://laendercode.net/fr/3-letter-list.html')
table = page.find('table')
liste3=ToDf(table)

In [13]:
liste2.to_csv(r'C:\Users\sylva\Desktop\git\Statap_cice\liste_pays_2lettres.csv')
liste3.to_csv(r'C:\Users\sylva\Desktop\git\Statap_cice\liste_pays_3lettres.csv')